# Emulating using GPs


In [1]:
! pip install --upgrade xarray zarr  cftime nc-time-axis #gcsfs
#PG for making maps
# a fix from https://github.com/SciTools/cartopy/issues/1893 
# because of a mismatch between Ubuntu vn and Cartopy required LIBGEOS :(
!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install -q -c conda-forge cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
✨🍰✨ Everything looks OK!

# All requested packages already installed.



In [2]:
!pip install cis scitools-iris gpflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
## Ignore my broken HDF5 install...
os.putenv("HDF5_DISABLE_VERSION_CHECK", '1')
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/CCIR-2022/CCIR-ESEm/ESEm//')
sys.path.append('/content/drive/MyDrive/CCIR-2022/CCIR-ESEm/ESEm/docs/examples/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#!pip install protobuf==3.2.0
#import iris
from examples_utils import get_bc_ppe_data

from esem import gp_model
from esem.utils import get_random_params


import matplotlib.pyplot as plt
%matplotlib inline

## Read in the parameters and observables

In [5]:
ppe_params, ppe_aaod = get_bc_ppe_data()

In [6]:
n_test = 5

X_test, X_train = ppe_params[:n_test], ppe_params[n_test:]
Y_test, Y_train = ppe_aaod[:n_test,0], ppe_aaod[n_test:,0]

## Setup and run the models

In [7]:
model = gp_model(X_train, Y_train)

In [8]:
model.train()

In [9]:
m, v = model.predict(X_test.values)

In [10]:
## validation_plot(Y_test.data.flatten(), m.data.flatten(), v.data.flatten())

In [ ]:
import iris
import iris.quickplot as qplt
qplt.pcolormesh((m.collapsed('sample', iris.analysis.MEAN)-Y_test.collapsed('job', iris.analysis.MEAN)), cmap='RdBu_r', vmin=-0.0001, vmax=0.0001)
plt.gca().coastlines()

In [18]:
## Note the model variance is constant across the outputs
v.data.max()

1.1356538598180312e-06

In [19]:
get_random_params(3, int(1e5)).shape

(100000, 3)

In [20]:
m, sd = model.batch_stats(get_random_params(3, int(1e3)))

  0%|          | 0/1000 [00:00<?, ?sample/s]

In [21]:
m, sd = model.batch_stats(get_random_params(3, int(1e4)), batch_size=10)

  0%|          | 0/10000 [00:00<?, ?sample/s]

In [ ]:
#m, sd = model.batch_stats(get_random_params(3, int(1e6)), batch_size=10000)

In [ ]:
https://stackoverflow.com/questions/70981856/how-do-i-fix-the-typeerror-load-missing-1-required-positional-argument-load 